In [2]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [3]:
role = sagemaker.get_execution_role()

In [ ]:
pytorch_estimator = PyTorch('SM_train_model.py',
                            instance_type='ml.m5.xlarge',
                            instance_count=1,
                            framework_version='1.12',
                            py_version='py38',
                            role=role,
                            source_dir = '../src/models/',
                            hyperparameters = {'epochs': 4, 'batch-size': 2048, 'embedding_dim': 256})
                            
pytorch_estimator.fit({'train': 's3://crystal-gazers/preprocessed/ca-2/'})

2022-10-12 19:53:57 Starting - Starting the training job...
2022-10-12 19:54:21 Starting - Preparing the instances for trainingProfilerReport-1665604437: InProgress
.........
2022-10-12 19:55:49 Downloading - Downloading input data...
2022-10-12 19:56:09 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-12 19:57:10,053 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-12 19:57:10,055 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-10-12 19:57:10,063 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-12 19:57:10,072 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-12 19:57:10,937 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-10-12 19:57:10,948 sagemaker-training-t

KeyboardInterrupt: 

In [ ]:
predictor = pytorch_estimator.deploy(instance_type='ml.m5.xlarge',
                                     initial_instance_count=1)


In [ ]:
response = predictor.predict(["una", "vaca", "vella", "per", "la", "muntanya"])

In [ ]:
print(response)

Inference

In [36]:
model_data = 's3://crystal-gazers/models/ca-100/model.tar.gz'
model = PyTorchModel(
    entry_point="inference.py",
    model_data=model_data,
    framework_version='1.12',
    py_version='py38',
    role=role,
    source_dir = '../src/models/'
)
                            

In [37]:
print(model)

In [38]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

-----!

In [39]:
dummy_data = {"input": ["la", "senyora", "vella", "per", "la", "muntanya"]}

In [40]:
res = predictor.predict(dummy_data)

In [41]:
print(res)

{'prediction': 'i'}


In [35]:
predictor.delete_endpoint()